In [2]:
from requests import post
from requests.exceptions import JSONDecodeError
import uuid
import os
from dotenv import load_dotenv

# Для ipynb явно загружаются переменные окружения
load_dotenv('.env')

GIGACHAT_AUTH_URL = os.environ.get("GIGACHAT_AUTH_URL")
GIGACHAT_AUTHORIZE_KEY = os.environ.get("GIGACHAT_AUTHORIZE_KEY")
GIGACHAT_REQUEST_URL = os.environ.get("GIGACHAT_REQUEST_URL")
GIGACHAT_MODEL = os.environ.get("GIGACHAT_MODEL")

if not GIGACHAT_REQUEST_URL:
    raise ValueError("GIGACHAT_REQUEST_URL не установлена в окружении")

if not GIGACHAT_MODEL:
    raise ValueError("GIGACHAT_MODEL не установлена в окружении")

if not GIGACHAT_AUTH_URL:
    raise ValueError("GIGACHAT_AUTH_URL не установлен в окружении")

if not GIGACHAT_AUTHORIZE_KEY:
    raise ValueError("GIGACHAT_AUTHORIZE_KEY не установлен в окружении")

# Обновлять токен доступа, если до окончания действия осталось 60 секунд
REFRESH_ACCESS_TOKEN_BEFORE_S = 60
__access_token = ""
__access_token_expires = ""

def refresh_access_token():
    """Refresh access_token for llm and set access_token_expires time

    Returns:
        Boolean: 
            True if refresh is success
            False if token is not refreshed
    """

    global __access_token
    global __access_token_expires

    headers = {
        "Content-Type": "application/x-www-form-urlencoded",
        "Accept": "application/json",
        "RqUID": f"{uuid.uuid4()}",
        "Authorization": f"Basic {GIGACHAT_AUTHORIZE_KEY}"
    }

    payload = {
        "scope": "GIGACHAT_API_PERS"
    }

    try:
        response = post(
            url=GIGACHAT_AUTH_URL,
            headers=headers,
            data=payload
        )

        response_json = response.json()

        __access_token = response_json.get("access_token")
        __access_token_expires = response_json.get("expires_at")

        if __access_token is None or __access_token_expires is None:
            print("Ошибка при выполнении запроса на обновление токена доступа",
                  "\nВ полученных данных отсутствуют 'access_token' или 'expires_at'",
                  f"\nОтвет сервиса: {response_json}")
            return False
    except JSONDecodeError as ex:
        print(f"Ошибка при парсинге ответа от сервиса при выполнении запроса на обновление токена доступа\n{ex}")
        return False
    except Exception as ex:
        print(f"Ошибка при выполнении запроса на обновление токена доступа\n{ex}")
        return False
    
    return True


In [3]:
is_refresh_success = refresh_access_token()
print(is_refresh_success)

True


In [4]:
from datetime import datetime

# При обращении к функции передачи сообщения нужна проверка действительности токена и его обновление

def refresh_access_token_if_expire(func):
    def wrapper(*args, **kwargs):
        # Проверка действительности токена
        # Преобразование в секунды для корректной работы с datetime
        token_expire_time_s = __access_token_expires / 1000.0
        token_expire_datetime = datetime.fromtimestamp(token_expire_time_s)
        # Остаток времени до истечения токена в секундах
        time_left_s = (token_expire_datetime - datetime.now()).total_seconds()

        print(
                "Refreshing access token",
                f"Expire datetime is: {token_expire_datetime}",
                f"Seconds left: {time_left_s}"
            )
        
        # Если действие токена скоро закончится или уже закончилось - обновить
        if time_left_s < REFRESH_ACCESS_TOKEN_BEFORE_S:
            print(
                "Refreshing access token",
                f"Expire datetime is: {token_expire_datetime}",
                f"Seconds left: {time_left_s}"
            )
            refresh_access_token()
        
        return func(*args, **kwargs)

    return wrapper

In [ ]:
# Нужна выгрузка истории сообщений по достижении определенного лимита, чтобы не хранить всё в оперативной памяти
# Нужно лимитировать историю сообщений, передаваемую LLM

# Формат хранения истории сообщений
# message_history = [
#     {
#         "role": "user",
#         "content": "request text"
#     }
# ]

import json

message_history = []

@refresh_access_token_if_expire
def bot_make_request(request_message: str):
    message_history.append({
        "role": "user",
        "content": request_message
    })

    headers = {
        "Content-Type": "application/json",
        "Accept": "application/json",
        "Authorization": f"Bearer {__access_token}"
    }

    body = json.dumps({
        "model": GIGACHAT_MODEL,
        "messages": message_history[-10:] # Последние 10 сообщений из истории
    })

    try:
        response = post(
            url=GIGACHAT_REQUEST_URL,
            headers=headers,
            data=body
        )

        response_json = response.json()

        assistant_message = response_json.get("choices")[0].get("message").get("content")

        if assistant_message is None:
            print(f"[ERROR] В полученных данных отсутствует сообщение модели")
            return False
        
        # Добавить ответ модели в историю сообщений
        message_history.append({
            "role": "assistant",
            "message": assistant_message
        })
    except JSONDecodeError as ex:
        print(f"[ERROR] Ошибка при парсинге ответа от сервиса при выполнении запроса на обновление токена доступа\n{ex}")
        return False
    except Exception as ex:
        print("[ERROR] Ошибка при выполнении запроса к LLM", ex)
        return False
    
    return True

In [17]:
bot_make_request("Привет! Какие у тебя возможности?")

Refreshing access token Expire datetime is: 2026-02-15 14:53:25.593000 Seconds left: 1344.974752
{'choices': [{'message': {'content': 'Привет!\n\nВот основные мои способности:\n\n- **Работа с текстом:** анализ, создание контента (статьи, рассказы, стихи), помощь в написании писем, сочинений, эссе, сценариев, диалогов и многое другое.\n- **Математика и логика:** решение уравнений, неравенств, логических задач, головоломок, подбор формул, пояснение сложных вопросов.\n- **Программирование:** разбор алгоритмов, написание программного кода, объяснение синтаксиса и структуры различных языков программирования (Python, JavaScript, C++, SQL и другие).\n- **Преподавание и обучение:** объяснения сложных понятий простым языком, составление учебных планов, подсказки по подготовке домашних заданий.\n- **Философия, психология, история, культура:** обсуждение любых философских, психологических и культурных вопросов.\n- **Чат и общение:** простое общение на любую тему, шутки, обсуждение новостей и собы

True

In [11]:
print(message_history)

[{'role': 'user', 'content': 'Привет! Какие у тебя возможности?'}, {'role': 'assistant', 'message': 'Вот основные возможности, которыми я обладаю:\n\n### 📚 Работа с текстом  \n- Генерирую тексты разных форматов: эссе, статьи, сценарии, стихи, письма и многое другое.\n- Могу редактировать, анализировать и исправлять ошибки в русском тексте.\n- Пишу пошаговые инструкции и объясняю сложные темы простым языком.\n- Решаю задачи по математике, физике, химии и другим предметам.\n- Анализирую факты, события и материалы, помогая разобраться в сложных вопросах.\n\n### 💡 Генерация идей и креативность  \n- Помогаю придумывать оригинальные идеи для бизнеса, творчества и повседневной жизни.\n- Предлагаю нестандартные решения проблем и задач.\n\n### 🤔 Логическое мышление и решение задач  \n- Решаю головоломки, задачи на рассуждение и аналитику.\n- Объясняю сложные концепции простыми словами.\n\n### 🎨 Возможности работы с дополнительными навыками (если активированы):  \n- Создаю изображения по вашему 

In [13]:
bot_make_request("Сгенерируй одну цитату для волка=)")

Refreshing access token Expire datetime is: 2026-02-15 14:53:25.593000 Seconds left: 1581.090878
BODY: {"model": "GigaChat-2", "messages": [{"role": "user", "content": "\u041f\u0440\u0438\u0432\u0435\u0442! \u041a\u0430\u043a\u0438\u0435 \u0443 \u0442\u0435\u0431\u044f \u0432\u043e\u0437\u043c\u043e\u0436\u043d\u043e\u0441\u0442\u0438?"}, {"role": "assistant", "message": "\u0412\u043e\u0442 \u043e\u0441\u043d\u043e\u0432\u043d\u044b\u0435 \u0432\u043e\u0437\u043c\u043e\u0436\u043d\u043e\u0441\u0442\u0438, \u043a\u043e\u0442\u043e\u0440\u044b\u043c\u0438 \u044f \u043e\u0431\u043b\u0430\u0434\u0430\u044e:\n\n### \ud83d\udcda \u0420\u0430\u0431\u043e\u0442\u0430 \u0441 \u0442\u0435\u043a\u0441\u0442\u043e\u043c  \n- \u0413\u0435\u043d\u0435\u0440\u0438\u0440\u0443\u044e \u0442\u0435\u043a\u0441\u0442\u044b \u0440\u0430\u0437\u043d\u044b\u0445 \u0444\u043e\u0440\u043c\u0430\u0442\u043e\u0432: \u044d\u0441\u0441\u0435, \u0441\u0442\u0430\u0442\u044c\u0438, \u0441\u0446\u0435\u043d\u0430\u04

True

In [18]:
from pathlib import Path

__username = None
__save_message_history = False

def bot_load_user_config(config_path="conf/user_settings.json"):
    global __username
    global __save_message_history

    path = Path(config_path)

    if (path.exists()):
        try:
            with open(path, "r", encoding="utf-8") as settings_file:
                settings = json.load(settings_file)

                __username = settings.get("username")
                __save_message_history = settings.get("save_message_history", False)

                if __username is None:
                    print(f"[ERROR] Не удается найти имя пользователя в файле конфигурации {config_path} (username)")
                    return False
                
                if __save_message_history is None:
                    print(f"[WARN] Флаг сохранения истории сообщений установлен в False, так как не был найден в файле конфигурации {config_path} (save_message_history)")
                
                return True
        except JSONDecodeError as ex:
            print("[ERROR] Ошибка при парсинге файла конфигурации в JSON", ex)
        except Exception as ex:
            print("[ERROR] Непредвиденная ошибка при чтении файла конфигурации", ex)
    else:
        print(f"[ERROR] Не удается найти файл конфигурации пользователя по пути {path}")
        return False

In [23]:
bot_load_user_config()

True

In [24]:
print(__username, __save_message_history)

host True


In [ ]:
from enum import Enum

class FileFormat(Enum):
    JSON = "json"
    TEXT = "txt"

# Функция для сохранения файла
def utils_save_file(path: Path, format: str, data):
    
    if not path.exists():
        pass

    if format == FileFormat.JSON:
        pass
    elif format == FileFormat.TEXT:
        pass
    else:
        print("[ERROR] Неизвестный формат файла. Сохранение не будет выполнено")

# Функция для сохранения истории сообщений
def bot_save_message_history(message_history: list):
    path = Path(f"data/history/{__username}.json")
    utils_save_file(path, FileFormat.JSON, message_history)

In [ ]:
# TODO

# Функция для генерации контекста на основе предыдущих сообщений
def bot_generate_context(message_history: list):
    pass

# Функция для сохранения контекста (под капотом использовать функцию сохранения файла)
def bot_save_context():
    pass